# DCGN

Deep learning approach for cancer subtype classification using high-dimensional gene expression data

# Import packages

In [ ]:
import numpy as np
import scipy.io
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,MaxPool2D,Dropout,Flatten,Dense
from tensorflow.keras import Model,datasets
import os

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

# Loading data

In [ ]:
mat = scipy.io.loadmat('/data/BRCASmote.mat')

In [ ]:
data = np.array(mat['data']).astype(np.float32)
data = data.T
tag = np.array(mat['targets']).astype(np.float32)

In [ ]:
data.shape

(4221, 20000)

In [ ]:
tag.shape

(4221, 1)

Output the retrieved data and labels, and make sure the data is (number of samples, feature dimension) and the labels are (number of samples, 1)

# Perform feature standardization
Feature normalization is performed on the gene expression data before model training so that all features in the dataset have zero mean and unit variance

In [ ]:
data.mean(axis=1)
data = data - data.mean(axis=0,keepdims=True)
data = data / np.sqrt(data.var(axis=0,keepdims = True))
data.var(axis=0)

array([1.0000001 , 0.99999964, 0.9999994 , ..., 0.9999986 , 1.0000017 ,
       1.0000012 ], dtype=float32)

# Set the random seed to a certain value for reproducibility and disrupt the data set

In [ ]:
np.random.seed(7)
np.random.shuffle(data)
np.random.seed(7)
np.random.shuffle(tag)
tf.random.set_seed(7)

Gelu activation function

In [ ]:
def gelu(x):
            return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

# Model
DCGN combining CNN and BiGRU, aiming to achieve nonlinear dimensionality reduction in the process of learning important features.

Ensure the  shape of the build function is (number of samples, feature dimension) and change the number of samples according to different data sets

In [ ]:
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow.keras.layers import Reshape
from tensorflow.keras import Input
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.layers import GRU,Conv2D,BatchNormalization,Activation,MaxPool2D,Dropout,Flatten,Dense
from tensorflow.keras import Model,datasets
import os

class MyModel(tf.keras.Model):
    

    def __init__(self):
        super (MyModel,self).__init__()
        self.f5 = tf.keras.layers.Dense(1024,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        
        self.conv1 = tf.keras.layers.Conv2D(filters = 128,kernel_size = 3,padding='same',strides = 2,activation=gelu)
        self.max1 = tf.keras.layers.MaxPool2D(pool_size = (2,2),strides = 2)
        self.l1 = tf.keras.layers.Bidirectional(GRU(64,return_sequences = True,bias_regularizer=tf.keras.regularizers.l2(1e-4),
                                         activity_regularizer=tf.keras.regularizers.l2(1e-5)))
        self.conv3 = tf.keras.layers.Conv2D(filters = 64,kernel_size = 3,padding='same',strides = 2,activation=gelu)
        self.max2 = tf.keras.layers.MaxPool2D(pool_size = (2,2),strides = 2)
        
        
        self.flatten = tf.keras.layers.Flatten()
        self.f1 = tf.keras.layers.Dense(128,activation =gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.f2 = tf.keras.layers.Dense(64,activation =gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.d2 = tf.keras.layers.Dropout(rate=0.6)
        self.f3 = tf.keras.layers.Dense(32,activation =gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.d3 = tf.keras.layers.Dropout(rate=0.7)
        self.f4 = tf.keras.layers.Dense(10)
        
    def call(self,x):
        x = self.f5(x)
        x = Reshape((32,32,1))(x)
        x = self.conv1(x)
        x = self.max1(x)
        x = Reshape((128,64))(x)
        x = self.l1(x)
        
        x = Reshape((128,128,1))(x)
        x = self.conv3(x)
        x = self.max2(x)

        x = self.flatten(x)
        x = self.f1(x)
        x = self.f2(x)
        x = self.d2(x)
        x = self.f3(x)
        x = self.d3(x)
        y = self.f4(x)
        return y
    
    
model = MyModel()
model.build(input_shape=(1010,20000))
model.call(Input(shape=(20000,)))

model.summary()

# Training and validation
Import multi-Category evaluation packages and set the loss function.Define gradient descent to minimize the classification loss during the training step

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
import os

optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
        y_pre = tf.argmax(logits,axis=1)
        y_pre = tf.reshape(y_pre,y.shape)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    f1 = f1_score(y, y_pre, average='weighted' )
    p = precision_score(y, y_pre, average='weighted')
    r = recall_score(y, y_pre, average='weighted')
    train_acc_metric.update_state(y, logits)
    return f1,p,r


def val_step(x, y):
    val_logits = model(x, training=False)
    val_y_pre =tf.argmax(val_logits,axis=1)
    val_y_pre =tf.reshape(val_y_pre,y.shape)
    f1 = f1_score(y, val_y_pre, average='weighted' )
    p = precision_score(y, val_y_pre, average='weighted')
    r = recall_score(y, val_y_pre, average='weighted')
    cm = confusion_matrix(y,val_y_pre)
    kappa = cohen_kappa_score(y,val_y_pre)
    ham = hamming_loss(y,val_y_pre)
    val_acc_metric.update_state(y, val_logits)
    return f1,p,r,kappa,ham

# Results
Divide the dataset, perform the training and validation steps in 100 epochs, and finally perform the testing step. Note that the size of different datasets is not the same. The code needs to be modified when changing datasets.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
epos = 100
train_ds = tf.data.Dataset.from_tensor_slices((data[:3376],tag[:3376])).batch(256)
val_ds = tf.data.Dataset.from_tensor_slices((data[3376:4198],tag[3376:4198])).batch(256)
test_x = data[4198:]
test_y = tag[4198:]
for epo in range(epos):
    print("\nStart of epoch %d" % (epo,))
    for x_batch_train, y_batch_train in train_ds:
        train_f1,train_pre,train_reca=train_step(x_batch_train, y_batch_train)
    train_acc = train_acc_metric.result()

    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_ds:
        val_f1,val_pre,val_reca,kappa,ham = val_step(x_batch_val, y_batch_val)
    val_acc = val_acc_metric.result()
    
    val_acc_metric.reset_states()
    print("Training acc over epoch: %.4f" % (float(train_acc),))
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Training pre over epoch: %.4f" % (float(train_pre),))
    print("Validation pre: %.4f" % (float(val_pre),))
    print("Training reca over epoch: %.4f" % (float(train_reca),))
    print("Validation reca: %.4f" % (float(val_reca),))
    print("Training f1 over epoch: %.4f" % (float(train_f1),))
    print("Validation f1: %.4f" % (float(val_f1),))
    print("Validation kappa: %.4f" % (float(kappa),))
    print("Validation 海明距离: %.4f" % (float(ham),))
#Define model checkpoint and save the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = '/weights.{epo:02d}.hdf5' , monitor='val_f1', verbose=1,save_weights_only=True，
                                                save_best_only=True, mode='max')

model.save_weights('/')#File path to save the best model weights
model.load_weights('/')#Load the weights saved above
test_pre = model.predict(test_x)
test_acc = train_acc_metric(test_y,test_pre)
print("The training process is over and the testing begins...")
print("Test acc over training: %.4f" % (float(test_acc),))

Start of epoch 38
Training acc over epoch: 0.9406
Validation acc: 0.9356
Training pre over epoch: 0.9519
Validation pre: 0.9363
Training reca over epoch: 0.9500
Validation reca: 0.9356
Training f1 over epoch: 0.9501
Validation f1: 0.9355
Validation kappa: 0.9033
Validation 海明距离: 0.0644

Start of epoch 39
Training acc over epoch: 0.9505
Validation acc: 0.9356
Training pre over epoch: 0.9766
Validation pre: 0.9363
Training reca over epoch: 0.9250
Validation reca: 0.9356
Training f1 over epoch: 0.9478
Validation f1: 0.9355
Validation kappa: 0.9033
Validation 海明距离: 0.0644

Start of epoch 40
Training acc over epoch: 0.9406
Validation acc: 0.9406
Training pre over epoch: 0.9766
Validation pre: 0.9413
Training reca over epoch: 0.9500
Validation reca: 0.9406
Training f1 over epoch: 0.9619
Validation f1: 0.9407
Validation kappa: 0.9107
Validation 海明距离: 0.0594